### Some tests

In [ ]:
import os
import pickle
import sys

In [ ]:
replay_files_path = '../data/replay_files'
replay_files = sorted(os.listdir(replay_files_path))
replay_files.pop(0)

In [ ]:
sample_file = replay_files[0]
with open(os.path.join(replay_files_path, sample_file), 'rb') as handle:
    episode_data = pickle.load(handle)

episode = episode_data['observations']
# sys.getsizeof(episode_data), sys.getsizeof(episode)
episode['ball'].dtype

In [ ]:
import numpy as np
a = [x for x in range(5)]
a_arr = np.array(a, dtype='float32')
np.append(a_arr, 6.0, 0)
# a_arr.append(6.0)
# a_arr[5] = 6.0
a_arr
# sys.getsizeof(a), a_arr.nbytes, sys.getsizeof(a_arr), a_arr.dtype, a_arr

### Create a dataframe with the top replays

In [ ]:
### !!!!!!!!!!!!!!!!!!! Do not delete this cell !!!!!!!!!!!!!!!!!!!
import os
from tqdm import tqdm
import pickle
import time
import numpy as np
from gfootball.env.wrappers import Simple115StateWrapper
import sys
# Here we will write the pickle files
def prepare_npy_dataset_from_replay_files(replay_files, replay_files_path):
    obs_save_dir = '/home/ssk/Study/GRP/dataset/top_480.csv'
#     replay_files_path = '../data/replay_files'

#     if not os.path.exists(obs_save_dir):
#         os.mkdir(obs_save_dir)
    replay_dict = {}
    for replay in tqdm(replay_files):
        with open(os.path.join(replay_files_path, replay), 'rb') as pkl_file:
            episode_data = pickle.load(pkl_file)

        episode_no = replay.split('.')[0]
        episode = episode_data['observations']
        episode['active'] = episode_data['players'][0]['active']
        episode_length = 3002
        raw_obs = {}

        episode_dir = os.path.join(obs_save_dir, episode_no)
#         if not os.path.exists(episode_dir):
#             os.mkdir(episode_dir)

        for step in range(episode_length):
            for (key, item) in episode.items():
                raw_obs[key] = item[step]

            float115_frame =  Simple115StateWrapper.convert_observation([raw_obs], True)[0].tolist()
            float115_frame_arr = np.array(float115_frame, dtype='float32')
            action = episode_data['players'][0]['action'][step]
            
            frame_name = episode_no+f'_{step}'
            if len(action) != 0:
                float115_frame.extend(action)
                fram_save_path = os.path.join(episode_dir, frame_name)
                replay_dict[frame_name] = np.array(float115_frame, dtype='float32')
#                 print(f"Size of the frame list : {sys.getsizeof(float115_frame)} bytes")
#                 print(f"Size of the numpy array for the same list : {np.array(float115_frame, dtype='float32').nbytes} bytes")
#                 np.save(fram_save_path, np.array(float115_frame))
    
    a_file = open("../data/replay_data.pkl", "wb")
    pickle.dump(replay_dict, a_file)
    a_file.close()
    print(f"Size of the replay dict for one file: {sys.getsizeof(replay_dict)/1024/1024} MB")
    print(len(replay_dict.keys()))

In [ ]:

replay_files_path = '../data/replay_files'
replay_files = sorted(os.listdir(replay_files_path))
replay_files.pop(0)
# replay_files = replay_files[0:1]
print(f"total replay files: {len(replay_files)}")
# replay_files = replay_files[0:1]

start = time.perf_counter()
prepare_npy_dataset_from_replay_files(replay_files[0:100], replay_files_path)
end =  time.perf_counter()

print(f"Total time needed to process {len(replay_files)}: {end-start}s")
print(f"Time needed to process a single file: {(end-start)/len(replay_files)}s")

In [ ]:
with open('../data/replay_data.pkl', 'rb') as handle:
    loaded_replay_dict = pickle.load(handle)

print(f"Size of the loaded dict: {sys.getsizeof(loaded_replay_dict)/1024/1024} MB")

### Create a dataframe with the top 480 files

In [ ]:
### !!!!!!!!!!!!!!!!!!! Do not delete this cell !!!!!!!!!!!!!!!!!!!
import os
from tqdm import tqdm
import pickle
import time
import numpy as np
from gfootball.env.wrappers import Simple115StateWrapper

# Here we will write the pickle files
def prepare_df_with_top_replays(top_replay_df):
    obs_save_dir = '/home/ssk/Study/GRP/dataset/top_480.csv'
    replay_files_path = '../data/replay_files'
    replay_dict = {}
    for idx, row in top_replay_df.iterrows():
        print(f"Working on {idx+1}/{480} files")
        episode_no = str(int(row[0]))
        episode_file_path = os.path.join(replay_files_path, f"{episode_no}.p")
        with open(episode_file_path, 'rb') as pkl_file:
            episode_data = pickle.load(pkl_file)

        episode = episode_data['observations']
        episode['active'] = episode_data['players'][0]['active']
        episode_length = 3002
        raw_obs = {}

        for step in range(episode_length):
            for (key, item) in episode.items():
                raw_obs[key] = item[step]

            float115_frame =  Simple115StateWrapper.convert_observation([raw_obs], True)[0].tolist()
            float115_frame_arr = np.array(float115_frame, dtype='float32')
            action = episode_data['players'][0]['action'][step]
            
            frame_name = episode_no+f'_{step}'
            if len(action) != 0:
                float115_frame.extend(action)
                replay_dict[frame_name] = np.array(float115_frame, dtype='float32')
#                 print(f"Size of the frame list : {sys.getsizeof(float115_frame)} bytes")
#                 print(f"Size of the numpy array for the same list : {np.array(float115_frame, dtype='float32').nbytes} bytes")
#                 np.save(fram_save_path, np.array(float115_frame))
    
    a_file = open("../data/top_480_replay_data.pkl", "wb")
    pickle.dump(replay_dict, a_file)
    a_file.close()
    print(f"Size of the replay dict: {sys.getsizeof(replay_dict)/1024/1024} MB")
#     print(len(replay_dict.keys()))



In [ ]:
import pandas as pd
top_df = pd.read_csv('../data/top_480_replay_files.csv', header=None)
prepare_df_with_top_replays(top_df)

### Main training loop

In [ ]:
import sys
sys.path.append("..")

import logging
import os
# import time
# import datetime
import random
import pandas as pd
import numpy as np
import torch
# import torch.nn.functional as F
import torch.nn as nn
# import torch.optim as optim
import csv
# import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# from tqdm import tqdm
# from os.path import join, dirname
# from sklearn.model_selection import train_test_split
from imitation_learning.dataset.frame_dataset_dict import Float115Dataset
from models.mlp_original import MLPModel
from utils.solver import Solver
# from gfootball.env.wrappers import Simple115StateWrapper
from utils.solver import Solver
from torch.utils.tensorboard import SummaryWriter
import pickle
# from imitation_learning.dataset.dict_dataset import Float115Dataset as DictDataset
# import hydra
# import wandb
# from omegaconf import DictConfig
# import torchtoolbox.transform as transforms
# import torchvision
# import pickle

In [ ]:
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

In [ ]:
# Setting the seeds for result reproducibility
os.environ['PYTHONHASHSEED'] = str(42)
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [ ]:
# Loading the dataset containing names of all the frames
logging.info("Reading the dataset")
with open('../data/top_480_replay_data.pkl', 'rb') as pkl_file:
    dict_dataset = pickle.load(pkl_file)
    dataset = pd.DataFrame(dict_dataset.keys())
#     print(len(dataset))
# dataset = pd.read_csv('../data/frames.csv', header=None)[0]
logging.info("Dataset loaded into the memory")

In [ ]:
# Creating Train, Val and Test Dataset
train, val, test = np.split(dataset.sample(frac=1, random_state=42), [
                                    int(.6 * len(dataset)), int(.8 * len(dataset))])
train, val, test = train.reset_index(drop=True), \
                                        val.reset_index(drop=True), \
                                        test.reset_index(drop=True)

In [ ]:
len(train), len(val), len(test)

In [ ]:
# dataset_path = '/home/ssk/Study/GRP/dataset/npy_files'
train_dataset, val_dataset, test_dataset = Float115Dataset(train, dict_dataset), \
                                            Float115Dataset(val, dict_dataset), \
                                            Float115Dataset(test, dict_dataset)

logging.info(f"Number of training samples: {len(train_dataset)}")
logging.info(f"Number of validation samples: {len(val_dataset)}")
logging.info(f"Number of test samples: {len(test_dataset)}")

In [ ]:
del(train)
del(val)
del(test)
del(dataset)
del(dict_dataset)
del(pkl_file)

In [ ]:
batch_size = 250
lr = 0.00001

In [ ]:
# Creating the dataloaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True, num_workers=1)


In [ ]:
# Loading the model and defining different parameters for the training
model = MLPModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# scheduler = ReduceLROnPlateau(optimizer=optimizer, mode='max', patience=1, verbose=True, factor=0.2)
criterion = nn.CrossEntropyLoss()

In [ ]:
writer = SummaryWriter()
solver = Solver(model, train_loader, val_loader, criterion, lr, optimizer, writer=writer)
solver.train(epochs=10)
writer.close()

In [ ]:
solver.best_val_loss

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(solver.train_loss_history, label='train_loss')
plt.plot(solver.val_loss_history, label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.legend()
# plt.show()
plt.savefig('imitation_480.jpg')

In [ ]:
!tensorboard --logdir=runs

### PyTorch Lightning

### Best trial config: {'hidden_size': 512, 'lr': 0.01061546492450981, 'batch_size': 32}

In [ ]:
import sys
sys.path.append("..")

import pytorch_lightning as pl
import torch
from models.mlp import MLPModel

In [ ]:
# print(f"Lightning version: {pl.__version__}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
hparams = {}
hparams['hidden_size'] = 512
hparams['lr'] = 0.0106
hparams['lr_decay_rate'] = 0.25
hparams['batch_size'] = 32
hparams['activation'] = 'ReLU'
model = MLPModel(hparams)
model = model.to(device)
model.prepare_data()

In [ ]:
# # https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html
# trainer = None

# from pytorch_lightning.callbacks import EarlyStopping

# early_stopping = EarlyStopping(
#     monitor='val_loss',
#     patience=5,
# )

# trainer = pl.Trainer(
#     weights_summary=None,
#     profiler=True,
#     max_epochs=20,
#     gpus=1,
#     early_stop_callback=early_stopping,
#     check_val_every_n_epoch=1,
# )

# trainer.fit(model)

In [ ]:
# from utils.Util import test_and_save
# test_and_save(model)

In [ ]:
# model.visualize_predictions(model.)

In [ ]:
# import pandas as pd
# import pickle
# import sys
# with open('/home/ssk/Study/GRP/dataset/top_480_replay_data.pkl', 'rb') as handle:
#     data = pickle.load(handle)
# print(f"Size of data: {sys.getsizeof(data)/1024/1024} MB")

### Hyperparameter optimization using Optuna

In [ ]:
import sys
sys.path.append("..")

import pytorch_lightning as pl
import torch
from models.mlp import MLPModel
from utils.Util import save_model, load_model, test_and_save
from pytorch_lightning.callbacks import EarlyStopping

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback

In [ ]:
from pytorch_lightning import Callback

class MetricsCallback(Callback):
    """PyTorch Lightning metric callback."""
    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
)


In [ ]:
def objective(trial):
    # as explained above, we'll use this callback to collect the validation accuracies
    metrics_callback = MetricsCallback()  
    
    # create a trainer
    trainer = pl.Trainer(
        #train_percent_check=1.0,
        #val_percent_check=1.0,
        logger=False,                                                                  # deactivate PL logging
        max_epochs=2,                                                                 # epochs
        gpus=1 if torch.cuda.is_available() else None,                                 # #gpus
#         callbacks=[metrics_callback],                                                  # save latest accuracy
#         early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="val_n_correct"), # early stopping
        early_stop_callback = early_stopping,
    )

    # here we sample the hyper params, similar as in our old random search
#     trial_hparams = {"hidden_size": trial.suggest_int("hidden_size", 100, 250, 10),
# #                      "n_layers": trial.suggest_int("n_layers", 1, 6),
# #                      "dropout_rate": trial.suggest_loguniform("dropout_rate", 1e-4, 5e-1),
#                      "activation": trial.suggest_categorical("activation", ('PReLU', 'ReLU', 'LeakyReLU')),
#                      "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
#                      "lr_decay_rate": trial.suggest_loguniform("lr_decay_rate", 1e-1, 5e-1),
#                      "batch_size": 250}
    trial_hparams = {"activation": trial.suggest_categorical("activation", ('PReLU', 'ReLU', 'LeakyReLU')),
                     "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
                     "lr_decay_rate": trial.suggest_loguniform("lr_decay_rate", 1e-1, 5e-1),
                     "batch_size": trial.suggest_int("batch_size", 100, 250, 10)}
    
    # create model from these hyper params and train it
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)
    model = MLPModel(trial_hparams)
    model = model.to(device)
    model.prepare_data()
    trainer.fit(model)

    # save model
    save_model(model, '{}.p'.format(trial.number), "checkpoints")
    del(model)
    # return validation accuracy from latest model, as that's what we want to minimize by our hyper param search
    return metrics_callback.metrics[-1]["val_acc"]

In [ ]:
pruner = optuna.pruners.NopPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=100, timeout=3600)